[View in Colaboratory](https://colab.research.google.com/github/s16397/ADD/blob/master/tmp.ipynb)

In [3]:
import os
import pandas as pd
import numpy as np

from keras.utils import np_utils

array = np.zeros([3,4])
array[0]


array([0., 0., 0., 0.])

In [4]:
from numpy import array
from numpy import argmax
from keras.utils import to_categorical
# define example
data = [3, 3, 2, 0, 3, 2, 2, 1, 0, 1]
data = array(data)
print(data)
# one hot encode
encoded = to_categorical(data)
print(encoded)
# invert encoding
inverted = argmax(encoded[1])
print(inverted)

[3 3 2 0 3 2 2 1 0 1]
[[0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]
3


In [173]:
df = pd.DataFrame({'input1': np.array([ 1.0, 2.0, 3.0, 4.0,100.0]) ,'input2': np.array([ 5.0, 6.0, 7.0, 8.0,100.0]), 'label': [0,1,2,1,0]})


def get_x_and_y_arrays_as_dict(df, w=3, k=1):
  
  max_offset = df.shape[0] - w - k + 1

  y_matrix = np_utils.to_categorical(df.label)
  print('Y MATRIX: ', y_matrix)
  unique_labels_num = len(np.unique(df.label))
  print('UNIQUE:', unique_labels_num)

  x_array = np.zeros([max_offset , w * 2])
  y_array = np.zeros([max_offset , unique_labels_num])
  print('zeros x shape: ',x_array.shape)
  print('zeros y shape: ',y_array.shape)

  for i in range(0, max_offset):
    tmp = df.iloc[i:i+w,0:2].values.flatten()
    x_array[i]=tmp
    y_index = i + w  + k -1
    y_array[i] = y_matrix[y_index]
  
  return {'x': x_array, 'y': y_array}


x_y = get_x_and_y_arrays_as_dict(df, w=2, k=1)
print("dict x shape")
print(x_y['x'].shape)
print("dict y shape")
print(x_y['y'].shape)


  

# print(type(y_matrix[0]))
# print(y_matrix[1])

# i=0
# for arr in tmp:
#   print(i)
#   i = i+1
#   for el in arr:
#     print(el.shape)
df.iloc[:,2]

Y MATRIX:  [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]
UNIQUE: 3
zeros x shape:  (3, 4)
zeros y shape:  (3, 3)
dict x shape
(3, 4)
dict y shape
(3, 3)


0    0
1    1
2    2
3    1
4    0
Name: label, dtype: int64

In [0]:
def get_csv_file_as_df(csv_filename):
  
  !if [ ! -d ADD ]; then git clone "https://github.com/s16397/ADD.git" ADD; fi
  
  add_dir = './ADD'
  csv_filepath = os.path.join(add_dir, csv_filename)
  
  # read file as df
  df = pd.read_csv(csv_filepath, header='infer')

  return df


def oblicz_roznice_kursu_norm(kurs_otwarcia, kurs_zamkniecia):
  diff = (kurs_zamkniecia / kurs_otwarcia) - 1
  if (diff > 0.01):
    return 2
  elif (diff < -0.01):
    return 1
  else:
    return 0

def preprocess_data(df, verbose=0):
  
  # roznica kursu: 2-wzrost, 1-spadek, 0-bez zmian
  df['roznica_kursu'] = df['Zamkniecie'] - df['Otwarcie']
  df['roznica_kursu_norm'] = df.apply(lambda row: oblicz_roznice_kursu_norm(row['Otwarcie'], row['Zamkniecie']),axis=1) 
  
  # find max value of: Otwarcie, Najwyzszy, Najnizszy, Zamkniecie
  max_stock_index = df.iloc[:,1:5].max().max()
  
  # find min value of: Otwarcie, Najwyzszy, Najnizszy, Zamkniecie
  min_stock_index = df.iloc[:,1:5].min().min()
  
  # find mean values for: Otwarcie, Najwyzszy, Najnizszy, Zamkniecie
  mean_values_serie = df.iloc[:,1:5].mean()
  
  # find max of: Wolumen
  max_volume = df.Wolumen.max()
  
  # find min of: Wolumen
  min_volume = df.Wolumen.min()
  
  # find avg of: Wolumen
  avg_volume = df.Wolumen.mean()
  
  if(verbose):
    print('**************** STATISTICS ****************')
    print('MAX(Otwarcie, Najwyzszy, Najnizszy, Zamkniecie)= ', max_stock_index)
    print('MIN(Otwarcie, Najwyzszy, Najnizszy, Zamkniecie)= ', min_stock_index)
    print('AVG(Otwarcie)=', mean_values_serie.loc['Otwarcie'])
    print('AVG(Najwyzszy)=', mean_values_serie.loc['Najwyzszy'])
    print('AVG(Najnizszy)=', mean_values_serie.loc['Najnizszy'])
    print('AVG(Zamkniecie)=', mean_values_serie.loc['Zamkniecie'])
    print('AVG(Otwarcie, Najwyzszy, Najnizszy, Zamkniecie)= ', mean_values_serie.mean())
    print('MAX(Wolumen)', max_volume)
    print('MIN(Wolumen)', min_volume)
    print('AVG(Wolumen)', avg_volume)
    print('**************** **************** ****************')
  
  # Normalize Otwarcie
  df.insert(6, 'Otwarcie_norm', df.Otwarcie.apply(
      lambda otwarcie: normalize_value_to_zero_one(otwarcie,
                                                   min_stock_index,
                                                   max_stock_index)))
  # Normalize Najwyzszy
  df.insert(7, 'Najwyzszy_norm', df.Najwyzszy.apply(
      lambda najwyzszy: normalize_value_to_zero_one(najwyzszy,
                                                   min_stock_index,
                                                   max_stock_index)))
  
  # Normalize Najnizszy
  df.insert(8, 'Najnizszy_norm', df.Najnizszy.apply(
      lambda najnizszy: normalize_value_to_zero_one(najnizszy,
                                                   min_stock_index,
                                                   max_stock_index)))
  
  # Normalize Zamkniecie
  df.insert(9, 'Zamkniecie_norm',df.Zamkniecie.apply(
      lambda zamkniecie: normalize_value_to_zero_one(zamkniecie,
                                                   min_stock_index,
                                                   max_stock_index)))
  # Normalize Wolumen
  df.insert(10, 'Wolumen_norm', df.Wolumen.apply(
      lambda wolumen: normalize_value_to_zero_one(wolumen,
                                                 min_volume,
                                                 max_volume)))
  
  return df

def normalize_value_to_zero_one(value, min_value, max_value):
  return (value - min_value) / (max_value - min_value)


In [33]:
df = get_csv_file_as_df(csv_filename='wig20_d-trening.csv')

pro = preprocess_data(df)

pro.head(10)

# processed_df = preprocess_data(df=df, verbose=0)

# processed_df.iloc[0:10,:]

one_index = pro.loc[pro['roznica_kursu_norm'] == 0]

one_index
# for idx in one_index:
#   print(idx)



,Data,Otwarcie,Najwyzszy,Najnizszy,Zamkniecie,Wolumen,Otwarcie_norm,Najwyzszy_norm,Najnizszy_norm,Zamkniecie_norm,Wolumen_norm,roznica_kursu,roznica_kursu_norm
1,2012-01-03,2203.07,2207.93,2184.10,2204.27,35639348,0.565558,0.570455,0.546441,0.566767,0.158137,1.20,0
2,2012-01-04,2197.87,2200.78,2160.82,2183.98,53626894,0.560318,0.563250,0.522981,0.546320,0.255717,-13.89,0
5,2012-01-10,2140.68,2164.57,2134.01,2138.06,26790974,0.502686,0.526760,0.495964,0.500045,0.110136,-2.62,0
6,2012-01-11,2151.91,2155.59,2119.32,2131.00,25183819,0.514002,0.517711,0.481161,0.492931,0.101418,-20.91,0
8,2012-01-13,2182.11,2192.63,2147.67,2166.38,16137326,0.544436,0.555037,0.509730,0.528584,0.052342,-15.73,0
10,2012-01-17,2209.14,2236.08,2207.21,2207.87,38577577,0.571675,0.598823,0.569730,0.570395,0.174077,-1.27,0
11,2012-01-18,2225.97,2235.94,2204.98,2228.09,29851535,0.588635,0.598682,0.567483,0.590771,0.126739,2.12,0
12,2012-01-19,2239.67,2244.38,2200.99,2244.38,35769956,0.602441,0.607187,0.563462,0.607187,0.158846,4.71,0
13,2012-01-20,2243.23,2261.02,2232.64,2261.02,41967272,0.606028,0.623956,0.595356,0.623956,0.192465,17.79,0
14,2012-01-23,2260.36,2274.91,2252.88,2265.75,25812477,0.623291,0.637953,0.615753,0.628722,0.104828,5.39,0


In [7]:
df = get_csv_file_as_df(csv_filename='wig20_d-trening.csv')
df.head(10)

,Data,Otwarcie,Najwyzszy,Najnizszy,Zamkniecie,Wolumen
0,2012-01-02,2150.60,2199.00,2143.87,2194.11,9743950
1,2012-01-03,2203.07,2207.93,2184.10,2204.27,35639348
2,2012-01-04,2197.87,2200.78,2160.82,2183.98,53626894
3,2012-01-05,2181.38,2181.38,2133.77,2157.43,19517064
4,2012-01-09,2164.20,2164.20,2126.71,2126.71,22781829
5,2012-01-10,2140.68,2164.57,2134.01,2138.06,26790974
6,2012-01-11,2151.91,2155.59,2119.32,2131.00,25183819
7,2012-01-12,2133.78,2183.29,2130.67,2166.99,25221052
8,2012-01-13,2182.11,2192.63,2147.67,2166.38,16137326
9,2012-01-16,2153.32,2185.09,2150.77,2182.94,17438271
